In [0]:
%run "../00-init/load-data"

In [0]:
# Configuration
# TODO: Update these values for your environment
catalog_name = "main"
schema_name = "dbrx_12daysofdemos"

In [0]:
# Drop the volume to start fresh
spark.sql(f"DROP VOLUME IF EXISTS {catalog_name}.{schema_name}.stream")

In [0]:
#Create the volume for streaming data
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.stream")

In [0]:
import time
from datetime import datetime

# Streaming configuration
source_table = f"{catalog_name}.{schema_name}.reindeer_telemetry"
target_volume_path = f"/Volumes/{catalog_name}/{schema_name}/stream/reindeer_telemetry"
interval_seconds = 3
max_iterations = 100

print(f"Starting data streaming from {source_table} to {target_volume_path}")
print(f"Writing interval: {interval_seconds} seconds")
print(f"Total iterations: {max_iterations}\n")

# Read the source table once
df = spark.read.table(source_table)
row_count = df.count()
print(f"Loaded {row_count} rows from source table\n")

# Write data at regular intervals
iteration = 0
while iteration < max_iterations:
    try:
        iteration += 1
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_path = f"{target_volume_path}/batch_{timestamp}.parquet"
        
        # Write data to volume
        df.write.mode("overwrite").parquet(file_path)
        
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{current_time}] Iteration {iteration}/{max_iterations}: Written {row_count} rows to {file_path}")
        
        # Wait before next write (skip on last iteration)
        if iteration < max_iterations:
            time.sleep(interval_seconds)
        
    except KeyboardInterrupt:
        print(f"\nStreaming stopped by user at iteration {iteration}")
        break
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        print(f"Retrying in {interval_seconds} seconds...")
        time.sleep(interval_seconds)

print(f"\nStreaming completed! Total iterations: {iteration}/{max_iterations}")